In [1]:
import models.model_pretrain as mp
dir(mp)


/home/admin12345/anaconda3/envs/ALBEF_EPIC/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['ALBEF',
 'BertConfig',
 'BertForMaskedLM',
 'F',
 'VisionTransformer',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'concat_all_gather',
 'interpolate_pos_embed',
 'math',
 'nn',
 'np',
 'partial',
 'random',
 'torch']

In [3]:
import torch
from ruamel.yaml import YAML
from models.model_pretrain import ALBEF
from transformers import BertTokenizer
from models.xbert import BertConfig, BertForMaskedLM
from PIL import Image
from torchvision import transforms

# dummy image and text
img = Image.open("examples/image0.jpg").convert("RGB")
transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
])
img_tensor = transform(img).unsqueeze(0)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_input = tokenizer("a cat sitting on a sofa", return_tensors="pt", padding='max_length', max_length=20, truncation=True)


yaml_loader = YAML(typ='rt')
with open('./configs/Pretrain.yaml', 'r') as f:
    config = yaml_loader.load(f)


model = ALBEF(config=config, text_encoder='bert-base-uncased', tokenizer=tokenizer, init_deit=True)
model.eval()

# Compute saliency
with torch.no_grad():
    alpha = model.compute_saliency(img_tensor, text_input)
print("✅ saliency shape:", alpha.shape)

# Simulate masking and ITC
num_mask = int(0.35 * text_input.input_ids.size(1))
mask_indices = torch.multinomial(alpha, num_samples=num_mask, replacement=False)
masked_text = text_input.input_ids.clone()
for b in range(masked_text.size(0)):
    masked_text[b, mask_indices[b]] = model.text_encoder.config.mask_token_id

aux_outputs = model.aux_lm(masked_text, labels=text_input.input_ids)
logits = aux_outputs.logits
probs = torch.softmax(logits, dim=-1)
sampled_ids = torch.multinomial(probs.view(-1, probs.size(-1)), 1).view(masked_text.size())
replaced_mask = (sampled_ids != text_input.input_ids) & (masked_text == model.text_encoder.config.mask_token_id)


loss_itc = model.forward_itc(img_tensor, sampled_ids, replaced_mask)
print("✅ ITC loss:", loss_itc.item())




reshape position embedding from 196 to 256
_IncompatibleKeys(missing_keys=[], unexpected_keys=['head.weight', 'head.bias'])


AttributeError: 'BertConfig' object has no attribute 'fusion_layer'